# **Setups**

## Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp


## Functions: agglutination model

In [ ]:
# Differential equations describing the relationship of components in the immunoagglutination system
def Agglutination4_5(t, z, ko1, kd1, ko2, kd2, ko3):
    # 4 components
    # 5 k parameters
    AbAg,Ab2Ag,Ab,Ag = z
    return [+ko1*Ab*Ag-kd1*AbAg-ko2*Ab*AbAg+kd2*Ab2Ag-ko3*Ag*AbAg, 
            +ko2*Ab*AbAg-kd2*Ab2Ag,
            -ko1*Ab*Ag+kd1*AbAg-ko2*Ab*AbAg+kd2*Ab2Ag,
            -ko1*Ab*Ag+kd1*AbAg-ko3*Ag*AbAg]


# Normalized agglutination versus antigen using two initial particle concentrations
def Agglu_2AbpConc(Args, 
                   AbConcs=[1,10], 
                   AgConcs=np.arange(0,40,step=1),
                   Timepoint=1,
                   TimeSpan=3,
                   dpi=100):
    
    Lins=200
    Tindex=int(Lins/TimeSpan*Timepoint)

    #Figure: two particle concentrations
    plt.figure(figsize=[4.5,3],dpi=dpi,layout="tight")
    plt.style.use('default')
    i=0
    Agglutination=[]
    for AgConc in AgConcs:
        sol = solve_ivp(Agglutination4_5, [0,TimeSpan], [0,0,AbConcs[i],AgConc], args=Args,
                    dense_output=True)
        t = np.linspace(0, TimeSpan, Lins)
        z = sol.sol(t)
        #Agglutination.append((z.T[Tindex][1]+z.T[Tindex][0])/AbConcs[i])
        Agglutination.append((z.T[Tindex][1])/AbConcs[i])
        #print(AgConc)
    plt.plot(AgConcs, Agglutination,linewidth=2)
    plt.ylim(0,max(Agglutination)+0.2)

    #plot the K parameters 
    FitParams = (r'$k_{on,1}$ = '+f'{Args[0]:5.2f}\n'
                 r'$k_{off,1}$ = '+f'{Args[1]:5.2f}\n'
                 r'$k_{on,2}$ = '+f'{Args[2]:5.2f}\n'
                 r'$k_{off,2}$ = '+f'{Args[3]:5.2f}\n'
                 r'$k_{on,3}$ = '+f'{Args[4]:5.2f}')
    bbox = dict(boxstyle='round', fc='#F1FDED', ec='#2ca02c')
    plt.text(max(AgConcs), max(Agglutination)+0.17, FitParams, fontsize='small', bbox=bbox, 
             ha='right', va="top")

    i=1
    Agglutination=[]
    for AgConc in AgConcs:
        sol = solve_ivp(Agglutination4_5, [0,TimeSpan], [0,0,AbConcs[i],AgConc], args=Args,
                    dense_output=True)
        t = np.linspace(0, TimeSpan, Lins)
        z = sol.sol(t)
        #Agglutination.append((z.T[Tindex][1]+z.T[Tindex][0])/AbConcs[i])
        Agglutination.append((z.T[Tindex][1])/AbConcs[i])
    plt.plot(AgConcs, Agglutination, linewidth=2)
    
    plt.xlabel('Antigen concentration')
    plt.ylabel('Normalized agglutination')
    plt.legend([str(AbConcs[0]), str(AbConcs[1])], loc=2,
               title='Particle conc.\n(relative value)', 
               title_fontsize='small',fontsize='small',
               framealpha=0)
    #plt.title('Agglutination Simulation')

    plt.show()
    plt.close()

# **K Parameter Search**
- 4 components
- 5 parameters

## Parameter grid

In [ ]:
# Each appropriate combination of k will give a plot of normalized agglutination 
# simulated with two antibody-particle concentrations

AbConcs=[5,10]
AgConcs=np.arange(0,25,step=0.5)


# Different grids for parameter search 
'''
Param_Grids={'Kon1': np.arange(4,10,step=2),
             'Koff1': np.arange(0.2,1,step=0.4),
             'Kon2': np.arange(1,2,step=0.5),
             'Koff2': np.arange(0.05,0.2, step=0.05),
             'Kon3': np.arange(1,2,step=0.5)}

'''
Param_Grids={'Kon1': [2, 6, 10],
             'Koff1': [0.1, 0.4, 1],
             'Kon2': [0.5, 1],
             'Koff2': [0.05, 0.5],
             'Kon3': [1, 2]}


ParamCombine=pd.DataFrame(columns=['Kon1','Koff1','Kon2','Koff2','Kon3'])
for Kon1 in Param_Grids['Kon1']:
    for Koff1 in Param_Grids['Koff1']:
        for Kon2 in Param_Grids['Kon2']:
            for Koff2 in Param_Grids['Koff2']:
                for Kon3 in Param_Grids['Kon3']:
                    if Kon1/Koff1>=Kon2/Koff2 and Kon1>Kon2 and Kon1>Kon3:
                        Args = [Kon1, Koff1, Kon2, Koff2, Kon3]
                        ParamCombine.loc[len(ParamCombine)] = Args
                        Agglu_2AbpConc(Args, 
                                       AbConcs=AbConcs, 
                                       AgConcs=AgConcs)

# show a table of paramters combination                        
display(ParamCombine)

## Figure: normalized agglutination

In [ ]:
# With one param combination, plot normalized agglutination and time dynamics figure 
# High resolution using dpi=600

Args=[6, 0.1, 1, 0.05, 2]
Agglu_2AbpConc(Args, AbConcs=[5,10], AgConcs=np.arange(0,25,step=0.1),
               Timepoint=1,dpi=600)

AbConc=10
AgConc=10
TimeSpan=3
Timepoint=1
Lins=200
Tindex=int(Lins/TimeSpan*Timepoint) #the index of timepoint

# Figures showing time-dependent dynamics of components in system
plt.style.use('seaborn-v0_8-pastel') #seaborn-v0_8-pastel is the light one
plt.figure(figsize=[2.5,2.5],dpi=600,layout="tight")
sol = solve_ivp(Agglutination4_5, [0,TimeSpan], [0,0,AbConc,AgConc], args=Args,
                dense_output=True)
t = np.linspace(0, 3, Lins)
z = sol.sol(t)
import matplotlib.pyplot as plt
plt.plot(t,z.T, linewidth=2, linestyle='-')
plt.xlim(0,1)
plt.ylim(0,10)
plt.xlabel('Time', fontsize='small')
plt.ylabel('Relative conc.', fontsize='small')
plt.legend(['Abp-Ag', 'Abp2-Ag', 'Abp', 'Ag'],fontsize='small',framealpha=0)
#plt.title('Agglutination')
plt.show()
plt.close()

## Figure: time-dependent dynamics

In [ ]:
# With selected antibody-particle concentrations and antigen selections,
# give example figures showing the time-dependent dynamics of each component in system
AbConcs=[5,10]
AgConcs=[1, 5, 10]

for AbConc in AbConcs:
    for AgConc in AgConcs:
        # Figures showing components over t
        plt.rcParams.update(plt.rcParamsDefault)
        plt.style.use('seaborn-v0_8-paper')
        plt.figure(figsize=[3,2.5],dpi=600,layout="tight",alpha=0)
        
        sol = solve_ivp(Agglutination4_5, [0,TimeSpan], [0,0,AbConc,AgConc], args=Args,
                        dense_output=True)
        t = np.linspace(0, 3, Lins)
        z = sol.sol(t)
        plt.plot(t,z.T, linewidth=2, linestyle='-')
        plt.xlim(0,2)
        plt.ylim(0,max(AbConc,AgConc))
        plt.xlabel('Time', fontsize='small')
        plt.ylabel('Relative conc.', fontsize='small')

        plt.legend(['Abp-Ag', 'Abp2-Ag', 'Abp (from '+str(AbConc)+')', 'Ag (from '+str(AgConc)+')'],
                   fontsize='small',framealpha=0)
        #plt.title('Agglutination')
        plt.show()
        plt.close()

## Save the parameter combination as csv

In [ ]:
ParamCombine.to_csv('ParamCombine.csv')